In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from typing import *
import os
import datetime
from mplcursors import cursor

from ipywidgets import widgets, interactive

%matplotlib widget

/usr/local/Caskroom/miniforge/base/envs/ptor/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
cd = os.getcwd()
idaho_data = pd.read_csv(cd + "/data/idaho_data/Idaho_2000_2022.csv", index_col = False)
idaho_data.head()

,Investment Name,Capital Committed,Unfunded Commitment,Paid-in Capital/Capital Drawn,Total Capital Distributed,Reported Market Value,Net IRR,Date,Capital Distributed Per Quarter
0,"Chisholm Partners IV, L.P.",25000000.0,17068716.0,7931284.0,3530.0,7504085.0,-0.0900,2000-03-31,3530.0
1,"Chisholm Partners IV, L.P.",25000000.0,13219281.0,11780719.0,1721.0,12590794.0,0.1484,2000-06-30,-1809.0
2,"Chisholm Partners IV, L.P.",25000000.0,9665879.0,15334121.0,1721.0,19480333.0,0.4699,2000-09-30,0.0
3,"Chisholm Partners IV, L.P.",25000000.0,9385743.0,15614257.0,480243.0,18168998.0,0.2344,2000-12-31,478522.0
4,"Chisholm Partners IV, L.P.",25000000.0,8646488.0,16353512.0,482051.0,18661104.0,0.1638,2001-03-31,1808.0


In [3]:
print(idaho_data.columns)
idaho_data.info()

Index(['Investment Name', 'Capital Committed', 'Unfunded Commitment',
       'Paid-in Capital/Capital Drawn', 'Total Capital Distributed',
       'Reported Market Value', 'Net IRR', 'Date',
       'Capital Distributed Per Quarter'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4332 entries, 0 to 4331
Data columns (total 9 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Investment Name                  4332 non-null   object 
 1   Capital Committed                4332 non-null   float64
 2   Unfunded Commitment              4332 non-null   float64
 3   Paid-in Capital/Capital Drawn    4332 non-null   float64
 4   Total Capital Distributed        4332 non-null   float64
 5   Reported Market Value            4332 non-null   float64
 6   Net IRR                          4332 non-null   float64
 7   Date                             4332 non-null   object 
 8   Capital Distri

In [4]:
idaho_data["Date"] = pd.to_datetime(idaho_data["Date"]).dt.date # this makes it play nice with the date picker

In [5]:
idaho_data["Date"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 4332 entries, 0 to 4331
Series name: Date
Non-Null Count  Dtype 
--------------  ----- 
4332 non-null   object
dtypes: object(1)
memory usage: 34.0+ KB


In [8]:
len(idaho_data)

4332

In [6]:
def makewidgets(df: pd.DataFrame = idaho_data, fund_col: str = "Investment Name", date_col: str = "Date"):
    """
    Makes the widgets for the fundplot
    """
    fund_names_widget = widgets.Dropdown(
    options = list(df[fund_col].unique()),
    value = "Chisholm Partners IV, L.P.",
    description = "Fund Name",
    disabled = False
    )

    start_year_widget = widgets.DatePicker(
        max = df[date_col].max(),
        min = df[date_col].min(),
        value = df[date_col].min(),
        disabled = False
    )
    end_year_widget = widgets.DatePicker(
        max = df[date_col].max(),
        min = df[date_col].min(),
        value = df[date_col].max(),
        disabled = False
    )
    return fund_names_widget, start_year_widget, end_year_widget


def fundplot(fund_names: widgets.Dropdown = None, start_year: widgets.Dropdown = None, end_year: widgets.Dropdown = None):
    """
    Makes an interactive plot with a fund dropdown and date filter
    
    inputs
    -------
    df:: pd.DataFrame, the relevant data

    outputs
    -------
    None
    """

    df = idaho_data.copy()

    df = df[df["Investment Name"] == fund_names]
    df = df[(df["Date"] >= start_year) & (df["Date"] <= end_year)]
    
    # don't really care about committment since its constant
    plots = df.columns.drop(["Investment Name", "Date", "Capital Committed"])

    if len(df) == 0:
        print("No selection")
    
    fig, ax = plt.subplots(nrows = 3, ncols = 2, figsize = (15,15))

    for i in range(3):
        for j in range(2):
            ax[i,j].plot(df[plots[2*i +j]])
            ax[i,j].set_title(plots[2*i + j])
            ax[i,j].set_xlabel("Quarter")
            ax[i,j].set_ylabel(plots[2*i + j])
            cursor(hover = True)

    fig.suptitle(f"Fund: {fund_names}, (n = {len(df)})", fontsize = 16)
    plt.show()


In [7]:
fund_name, start_year, end_year = makewidgets()
interactive(fundplot, fund_names = fund_name, start_year = start_year, end_year = end_year)

interactive(children=(Dropdown(description='Fund Name', options=('Chisholm Partners IV, L.P.', 'FCI Enclosures…